In [2]:
# Standard libraries
import pandas as pd
import numpy as np
from scipy import stats
# Visualization
import matplotlib.pyplot as plt
import wrds
# import os 
# os.system("pip install wrds")

In [ ]:
# Resources for WRDS
# WorldScope (I think this is what we want): 
#     https://wrds-web.wharton.upenn.edu/wrds//ds/tfn/ws/stock/index.cfm#variablesTab
#     https://wrds-web.wharton.upenn.edu/wrds//ds/compd/secm/index.cfm
# Use: 5001 -- Market Price - Year End, stuff like 5015 -- End Market Price - January, 5601 -- ticker

# CRSP also works but I think WorldScope is better.
# https://wrds-web.wharton.upenn.edu/wrds//ds/crsp/stock_a/msf.cfm
# https://wrds-web.wharton.upenn.edu/wrds/tools/variable.cfm?library_id=20&file_id=123976
# https://wrds-web.wharton.upenn.edu/wrds/tools/variable.cfm?library_id=20&file_id=123976

In [3]:
db = wrds.Connection()

Enter your WRDS username [gabri]:gnaval
Enter your password:········
WRDS recommends setting up a .pgpass file.
You can find more info here:
https://www.postgresql.org/docs/9.5/static/libpq-pgpass.html.
Loading library list...
Done


In [5]:
# db.list_tables('crsp')

In [24]:
db.get_table(library = 'crsp', table ='msf', obs = 10)

,cusip,permno,permco,issuno,hexcd,hsiccd,date,bidlo,askhi,prc,vol,ret,bid,ask,shrout,cfacpr,cfacshr,altprc,spread,altprcdt,retx
0,68391610,10000.0,7952.0,10396.0,3.0,3990.0,1985-12-31,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,-2.562500,NaN,1986-01-07,NaN
1,68391610,10000.0,7952.0,10396.0,3.0,3990.0,1986-01-31,-2.50000,-4.437500,-4.375000,1771.0,NaN,None,None,3680.0,1.0,1.0,-4.375000,0.25000,1986-01-31,NaN
2,68391610,10000.0,7952.0,10396.0,3.0,3990.0,1986-02-28,-3.25000,-4.375000,-3.250000,828.0,-0.257143,None,None,3680.0,1.0,1.0,-3.250000,0.25000,1986-02-28,-0.257143
3,68391610,10000.0,7952.0,10396.0,3.0,3990.0,1986-03-31,-3.25000,-4.437500,-4.437500,1078.0,0.365385,None,None,3680.0,1.0,1.0,-4.437500,0.12500,1986-03-31,0.365385
4,68391610,10000.0,7952.0,10396.0,3.0,3990.0,1986-04-30,-4.00000,-4.312500,-4.000000,957.0,-0.098592,None,None,3793.0,1.0,1.0,-4.000000,0.25000,1986-04-30,-0.098592
5,68391610,10000.0,7952.0,10396.0,3.0,3990.0,1986-05-30,-3.06250,-4.218750,-3.109375,1074.0,-0.222656,None,None,3793.0,1.0,1.0,-3.109375,0.09375,1986-05-30,-0.222656
6,68391610,10000.0,7952.0,10396.0,3.0,3990.0,1986-06-30,-2.90625,-3.296875,-3.093750,1069.0,-0.005025,None,None,3793.0,1.0,1.0,-3.093750,0.06250,1986-06-30,-0.005025
7,68391610,10000.0,7952.0,10396.0,3.0,3990.0,1986-07-31,-2.59375,-3.437500,-2.843750,1163.0,-0.080808,None,None,3793.0,1.0,1.0,-2.843750,0.06250,1986-07-31,-0.080808
8,68391610,10000.0,7952.0,10396.0,3.0,3990.0,1986-08-29,-1.03125,-2.625000,-1.093750,3049.0,-0.615385,None,None,3793.0,1.0,1.0,-1.093750,0.06250,1986-08-29,-0.615385
9,68391610,10000.0,7952.0,10396.0,3.0,3990.0,1986-09-30,-0.96875,-1.281250,-1.031250,3551.0,-0.057143,None,None,3793.0,1.0,1.0,-1.031250,0.06250,1986-09-30,-0.057143


In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# data_5rows = db.get_table(library='wrdssec', table='wrds_nlpsa', columns = ['gvkey', 'cik', 'filename', 'finterms_litigious_count'], obs=5)


# sql_query="""
# SELECT ...
# FROM ...
# """
# data_query = db.raw_sql(sql_query)

# This crsp thing works
db.get_table(library = 'crsp', table='saz_agg_ann', obs = 200)


In [14]:
# PRC Dataset 
PRC_df = pd.read_csv("../data/prc.csv" )

# Data Cleaning
PRC_df.drop(PRC_df.columns[[13,14,15]], axis=1, inplace = True)
PRC_df = PRC_df[PRC_df['Total Records'].notna()]

def records_to_int(record_str):
    return int(str(record_str).replace(",", ""))

PRC_df['Total Records'] = PRC_df['Total Records'].apply(records_to_int)

# Drop rows with 0 total records (Maybe toggle this off we want to fill in data)
PRC_df = PRC_df[PRC_df['Total Records'] != 0]

In [ ]:
# TODO LIST:
# What to do with 0 total records? - drop or use some averaging technique?
# Combine this dataset with another one - make sure no repeats
# Use some API to gather more info on these companies

# Things to explore:
# How do different type of attacks vary

In [15]:
# Audit Analytics Dataset
xls = pd.ExcelFile('../data/audit_analytics.xlsx')
aa_records_df = pd.read_excel(xls, 'PublicCyber')
aa_ddos_df = pd.read_excel(xls, 'DDoS')
aa_records_df

,Company name,Related party,Target relationship to Parent,CIK,Ticker,Market,IRS Number,Parent CIK,Parent Name,Bus Street 1,...,Section,Filing Date,Filing Disclosure,Cost/Estimate,Reimbursements,Insurance,Litigation Docket,Lead Defendant Representation,Lead Plaintiff Representation,Litigation Settlement
0,HUMANA INC,HUMANA INC,Parent,49071,HUM,NYSE,61-0647538,NaN,NaN,500 W MAIN ST,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,HERBALIFE NUTRITION LTD.,Herbalife Nutrition,Parent,1180262,HLF,NYSE,98-0377871,NaN,NaN,P.O. BOX 309GT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"Ultra Clean Holdings, Inc.","Ultra Clean Holdings, Inc.",Parent,1275014,UCTT,Nasdaq Global Market,61-1430858,NaN,NaN,150 INDEPENDENCE DRIVE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,KROGER CO,Kroger Pharmacy,Parent,56873,KR,NYSE,31-0345740,NaN,NaN,1014 VINE ST,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,CINTAS CORP,Cintas Corporation,Parent,723254,CTAS,Nasdaq Global Market,31-1188630,NaN,NaN,6800 CINTAS BLVD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
864,NaN,CareFirst BlueCross BlueShield,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
865,"RLJ ENTERTAINMENT, INC.","RLJ ENTERTAINMENT, INC.",Parent,1546381,NaN,NaN,45-4950432,NaN,NaN,8515 GEORGIA AVENUE,...,Controls,2015-05-08 00:00:00,The Company’s third-party US content distribu...,0,NaN,NaN,NaN,NaN,NaN,NaN
866,"Endurance International Group Holdings, Inc.","Bluehost, Justhost, Hostgator, Hostmonster, Fa...",Third-party,1237746,NaN,NaN,46-3044956,NaN,NaN,10 CORPORATE DRIVE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
867,WEYERHAEUSER CO,Premera Blue Cross,Third-party,106535,WY,NYSE,91-0470860,NaN,NaN,220 OCCIDENTAL AVENUE SOUTH,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
# Ticker
ticker_df = pd.read_csv("../data/ticker.csv" )
ticker_df

,Unnamed: 0,cik_str,ticker,title
0,0,320193,AAPL,Apple Inc.
1,1,789019,MSFT,MICROSOFT CORP
2,2,1018724,AMZN,AMAZON COM INC
3,3,1652044,GOOG,Alphabet Inc.
4,4,1293451,TCEHY,Tencent Holdings Ltd
...,...,...,...,...
11351,11351,1830081,CFVIU,CF Acquisition Corp. VI
11352,11352,1830063,NAACU,North Atlantic Acquisition Corp
11353,11353,1829889,IIIIW,INSU ACQUISITION CORP III
11354,11354,1829999,FOXWU,FoxWayne Enterprises Acquisition Corp.
